In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from deepctr.models import DeepFM,DIN
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
import numpy as np
import tensorflow as tf
import random

In [ ]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
  
def load_variable(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [67]:
pip install --upgrade google-cloud-bigquery

    100% |████████████████████████████████| 204kB 2.8MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/66/61/680d8a5a146e33ac463e0fa800ee37c6becbeee4aa02522fa6632dca8831/grpcio-1.39.0-cp37-cp37m-manylinux2010_x86_64.whl
    100% |████████████████████████████████| 81kB 4.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/3c/77/e2362b676dc5008d81be423070dd9577fa03be5da2ba1105811900fda546/packaging-21.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8c/72/6f3f4cdc5bb0294f8d7f3f8aacb617b4c3cb17554ed78f7e28009162c795/proto_plus-1.19.0-py3-none-any.whl
    100% |████████████████████████████████| 102kB 4.4MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8f/9e/a45476e0fda1ed17e8cc926f4c835e7ae1e2b6460dd00c35a4414ff5ecfa/google_crc32c-1.1.2-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-

In [2]:
#4月的
df1 = pd.read_csv('/home/jovyan/dataset/april_csvfile/0401_0410.csv')
df2 = pd.read_csv('/home/jovyan/dataset/april_csvfile/0411_0420.csv')
df3 = pd.read_csv('/home/jovyan/dataset/april_csvfile/0421_0430.csv')
#5月的
df4 = pd.read_csv('/home/jovyan/dataset/0501_0505.csv')
len(df1),len(df2),len(df3),len(df4)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(4162617, 4439353, 3848133, 2049848)

In [3]:
def whichdate(dates):
    if dates == '2021-05-01':
        return 51
    elif dates == '2021-05-02':
        return 52
    elif dates == '2021-05-03':
        return 53
    elif dates == '2021-05-04':
        return 54
    else:
        return 55

In [4]:
df4 = df4.sort_values('api_logtime')
df4_month = df4.api_logtime.apply(lambda x:whichdate(x.split(' ')[0]))
df4['month'] = df4_month#df4是5月的df

In [5]:
dfm4 = pd.concat([df1,df2,df3], ignore_index=True)#dfm4 --> 4月的df
month1 = len(dfm4)*[4]
dfm4['month'] = month1
df = pd.concat([dfm4,df4],ignore_index=True)# 合併4月和5月的df
df['uid'].fillna("unknow", inplace = True)#uid是空的填上unknow

In [10]:
lbe = LabelEncoder()
for i in ['gid']:
    df['gid_encode'] = lbe.fit_transform(df[i])
    df['gid_encode'] = df['gid_encode'].apply(lambda x:x+1)
#產生gid和對應id的dictionary
temp = df[['gid','gid_encode']].sort_values('gid_encode')
gid_encode_unique = temp.gid_encode.unique()
gid_unique = temp.gid.unique()
gid_dict = {}
for i in zip(gid_encode_unique,gid_unique):
    gid_dict[i[1]] = i[0]

In [62]:
df_uid = df[df.uid !='unknow']
df_ven_guid = df[df.uid == 'unknow']
df_uid = df_uid.drop(columns = ['ven_guid','ven_session'])
df_ven_guid = df_ven_guid.drop(columns = ['uid'])
df_uid = df_uid.sort_values('api_logtime')
df_ven_guid = df_ven_guid.sort_values('api_logtime')

In [124]:
df_uid_4 = df_uid[df_uid.month == 4]
df_uid_5 = df_uid[df_uid.month != 4]
df_ven_guid_4 = df_ven_guid[df_ven_guid.month == 4]
df_ven_guid_5 = df_ven_guid[df_ven_guid.month != 4]

In [125]:
len(df_uid_4),len(df_uid_5),len(df_ven_guid_4),len(df_ven_guid_5)

(3792823, 613520, 8657280, 1436328)

In [126]:
def uid_labelencode(df):
    for i in ['uid']:
        df[i] = lbe.fit_transform(df[i])
        df[i] = df[i].apply(lambda x:x+1)
def ven_guid_labelencode(df):
    for i in ['ven_guid']:
        df[i] = lbe.fit_transform(df[i])
        df[i] = df[i].apply(lambda x:x+1)

In [127]:
uid_labelencode(df_uid_4)
uid_labelencode(df_uid_5)
ven_guid_labelencode(df_ven_guid_4)
ven_guid_labelencode(df_ven_guid_5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [130]:
#刪掉點擊商品小於三個的uid或ven_guid
df_uid_4 = df_uid_4[df_uid_4['uid'].map(df_uid_4.uid.value_counts())>=3]
df_uid_5 = df_uid_5[df_uid_5['uid'].map(df_uid_5.uid.value_counts())>=3]
df_ven_guid_4 = df_ven_guid_4[df_ven_guid_4['ven_guid'].map(df_ven_guid_4.ven_guid.value_counts())>=3]
df_ven_guid_5 = df_ven_guid_5[df_ven_guid_5['ven_guid'].map(df_ven_guid_5.ven_guid.value_counts())>=3]

In [68]:
#計算每個商品在所有商品中(4/1-5/5)的出現頻率
under = 0
freq_list = []
freq = df.gid_encode.value_counts()
for i in freq.values:
    under += np.log10(i+1)
for i in freq.values:
    freq_list.append(np.log10(i+1)/under)

In [ ]:
import random
def create_set_byidname(df,idname):
    t_set = []
    neg_list = random.choices(freq.index,weights = freq_list, k=1000000)
    for userid, hist in df.groupby(idname):
        pos_list = hist['gid_encode'].tolist()
        if len(pos_list) > 101:#如果pos_list(clickstream)長度大於100，就取後面100筆
            pos_list = pos_list[-101:]
        def gen_neg():
            count = 0
            neg = pos_list[0]
            while neg in pos_list:
                if count == 100:#如果抽100次的負面樣本都抽到在pos_list的,就把count歸0
                    count = 0
                neg = random.choices(neg_list, k=1)[0]
                count += 1
            return neg
        neg_list2 = [gen_neg() for i in range(len(pos_list))]
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] 
            t_set.append((userid, hist, pos_list[i], 1))
            t_set.append((userid, hist, neg_list2[i], 0))
    return t_set
df_train_uid = create_set_byidname(df_uid_4,'uid')
df_train_venguid = create_set_byidname(df_ven_guid_4,'ven_guid')
train_set = df_train_venguid + df_train_uid
np.random.shuffle(train_set)
len(df_train_venguid),len(df_train_uid)

In [71]:
def create_array(data):
    g_id = []
    hist_g_id = []
    click = []
    seq_length = []

    for index, value in enumerate(data):
        hist_g_id.append(value[1])
        seq_length.append(len(value[1]))
        g_id.append(value[2])
        click.append(value[3])  
    for i in range(len(hist_g_id)):
        if len(hist_g_id[i]) != 100:
            hist_g_id[i] += (100-len(hist_g_id[i]))*[0]

    g_id = np.array(g_id)
    hist_g_id = np.array(hist_g_id)
    click = np.array(click)
    seq_length = np.array(seq_length)
    return g_id, hist_g_id, click, seq_length

In [72]:
g_id_train, hist_g_id_train, click_train, seq_length_train = create_array(train_set)

In [151]:
g_id_train, hist_g_id_train, click_train, seq_length_train

(array([1824365, 1278717,  936515, ...,  613386, 1055567,  378817]),
 array([[ 149969,  972274,  972274, ...,       0,       0,       0],
        [ 250176,  836563, 1444467, ...,       0,       0,       0],
        [1216868, 1216868, 1460994, ...,       0,       0,       0],
        ...,
        [1359690, 1359690,  496823, ...,       0,       0,       0],
        [1224981, 1224981, 1224981, ...,       0,       0,       0],
        [1053905, 1773460, 1053905, ...,       0,       0,       0]]),
 array([1, 0, 0, ..., 1, 1, 0]),
 array([20,  6,  6, ..., 53,  5,  3]))

In [75]:
behavior_feature_list = ["g_id"]
feature_columns = [SparseFeat('g_id', df.gid_encode.max()+1, embedding_dim=8)]
feature_columns += [VarLenSparseFeat(SparseFeat('hist_g_id',vocabulary_size=df.gid_encode.max()+1, embedding_dim=8, embedding_name='g_id'), maxlen = 100, length_name='seq_length')]
x = {'g_id': g_id_train, 
    'hist_g_id': hist_g_id_train, 
    'seq_length': seq_length_train}
y = click_train

In [76]:
for i, v in x.items():
    print(i)
    print(v.shape)

g_id
(14259236,)
hist_g_id
(14259236, 100)
seq_length
(14259236,)


In [37]:
model = DIN(feature_columns, behavior_feature_list)
model.compile('SGD', 'binary_crossentropy',metrics=['accuracy','AUC'])
checkpoint_path = "/home/jovyan/dataset/checkpoint_folder/model_401_430/rakuten_0401_0430.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1) 
history = model.fit(x, y, batch_size=128, verbose=1, epochs=10, callbacks=[cp_callback])

Epoch 1/10
111401/111401 [==============================] - 26833s 241ms/step - loss: 0.6617 - accuracy: 0.5709 - auc: 0.6167

Epoch 00001: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 2/10
111401/111401 [==============================] - 27634s 248ms/step - loss: 0.5416 - accuracy: 0.7302 - auc: 0.7958

Epoch 00002: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 3/10
111401/111401 [==============================] - 27802s 250ms/step - loss: 0.4727 - accuracy: 0.7887 - auc: 0.8534

Epoch 00003: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 4/10
111401/111401 [==============================] - 27839s 250ms/step - loss: 0.4215 - accuracy: 0.8210 - auc: 0.8867

Epoch 00004: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch 5/10
111401/111401 [==============================] - 28679s 257ms/step - loss: 0.3788 - accuracy: 0.8430 - auc: 0.9104

Epoch 00005: saving model to /home/jovyan/dataset/rakuten_0811_cp1.ckpt
Epoch

#### 15epoch

In [78]:
import keras
model = keras.models.load_model("/home/jovyan/dataset/rakuten_model_401_430")
checkpoint_path = "/home/jovyan/dataset/15epoch/401_430.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,verbose=1)
history = model.fit(x, y, batch_size=128, verbose=1, epochs=5, callbacks=[cp_callback])

Epoch 1/5
111401/111401 [==============================] - 28619s 257ms/step - loss: 0.3826 - accuracy: 0.8253 - auc: 0.9074

Epoch 00001: saving model to /home/jovyan/dataset/15epoch/401_430.ckpt
Epoch 2/5
111401/111401 [==============================] - 26982s 242ms/step - loss: 0.3202 - accuracy: 0.8616 - auc: 0.9370

Epoch 00002: saving model to /home/jovyan/dataset/15epoch/401_430.ckpt
Epoch 3/5
111401/111401 [==============================] - 26730s 240ms/step - loss: 0.2820 - accuracy: 0.8819 - auc: 0.9519

Epoch 00003: saving model to /home/jovyan/dataset/15epoch/401_430.ckpt
Epoch 4/5
111401/111401 [==============================] - 27007s 242ms/step - loss: 0.2547 - accuracy: 0.8953 - auc: 0.9612

Epoch 00004: saving model to /home/jovyan/dataset/15epoch/401_430.ckpt
Epoch 5/5
111401/111401 [==============================] - 27036s 243ms/step - loss: 0.2342 - accuracy: 0.9051 - auc: 0.9674

Epoch 00005: saving model to /home/jovyan/dataset/15epoch/401_430.ckpt
